In [1]:
import pandas as pd
import glob
import torch, os
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast, TrainingArguments, Trainer, DistilBertForSequenceClassification, DistilBertTokenizerFast
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

d:\AI_Dev_Vault\Anaconda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
d:\AI_Dev_Vault\Anaconda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU')
else:
    device = torch.device("cpu")
    print('CPU')

GPU


In [3]:
train_paths = ['./content/train/california_wildfires_2018_train.tsv',
               './content/train/canada_wildfires_2016_train.tsv',
               './content/train/cyclone_idai_2019_train.tsv',
               './content/train/ecuador_earthquake_2016_train.tsv',
               './content/train/greece_wildfires_2018_train.tsv',
               './content/train/hurricane_dorian_2019_train.tsv',
               './content/train/hurricane_florence_2018_train.tsv',
               './content/train/hurricane_harvey_2017_train.tsv',
               './content/train/hurricane_irma_2017_train.tsv',
               './content/train/hurricane_maria_2017_train.tsv',
               './content/train/hurricane_matthew_2016_train.tsv',
               './content/train/italy_earthquake_aug_2016_train.tsv',
               './content/train/kaikoura_earthquake_2016_train.tsv',
               './content/train/kerala_floods_2018_train.tsv',
               './content/train/maryland_floods_2018_train.tsv',
               './content/train/midwestern_us_floods_2019_train.tsv',
               './content/train/pakistan_earthquake_2019_train.tsv',
               './content/train/puebla_mexico_earthquake_2017_train.tsv',
               './content/train/srilanka_floods_2017_train.tsv']

val_paths = ['./content/dev/california_wildfires_2018_dev.tsv',
             './content/dev/canada_wildfires_2016_dev.tsv',
             './content/dev/cyclone_idai_2019_dev.tsv',
             './content/dev/ecuador_earthquake_2016_dev.tsv',
             './content/dev/greece_wildfires_2018_dev.tsv',
             './content/dev/hurricane_dorian_2019_dev.tsv',
             './content/dev/hurricane_florence_2018_dev.tsv',
             './content/dev/hurricane_harvey_2017_dev.tsv',
             './content/dev/hurricane_irma_2017_dev.tsv',
             './content/dev/hurricane_maria_2017_dev.tsv',
             './content/dev/hurricane_matthew_2016_dev.tsv',
             './content/dev/italy_earthquake_aug_2016_dev.tsv',
             './content/dev/kaikoura_earthquake_2016_dev.tsv',
             './content/dev/kerala_floods_2018_dev.tsv',
             './content/dev/maryland_floods_2018_dev.tsv',
             './content/dev/midwestern_us_floods_2019_dev.tsv',
             './content/dev/pakistan_earthquake_2019_dev.tsv',
             './content/dev/puebla_mexico_earthquake_2017_dev.tsv',
             './content/dev/srilanka_floods_2017_dev.tsv']

test_paths = ['./content/test/california_wildfires_2018_test.tsv',
              './content/test/canada_wildfires_2016_test.tsv',
              './content/test/cyclone_idai_2019_test.tsv',
              './content/test/ecuador_earthquake_2016_test.tsv',
              './content/test/greece_wildfires_2018_test.tsv',
              './content/test/hurricane_dorian_2019_test.tsv',
              './content/test/hurricane_florence_2018_test.tsv',
              './content/test/hurricane_harvey_2017_test.tsv',
              './content/test/hurricane_irma_2017_test.tsv',
              './content/test/hurricane_maria_2017_test.tsv',
              './content/test/hurricane_matthew_2016_test.tsv',
              './content/test/italy_earthquake_aug_2016_test.tsv',
              './content/test/kaikoura_earthquake_2016_test.tsv',
              './content/test/kerala_floods_2018_test.tsv',
              './content/test/maryland_floods_2018_test.tsv',
              './content/test/midwestern_us_floods_2019_test.tsv',
              './content/test/pakistan_earthquake_2019_test.tsv',
              './content/test/puebla_mexico_earthquake_2017_test.tsv',
              './content/test/srilanka_floods_2017_test.tsv']

train_file_paths = []

for path in train_paths:
    files = glob.glob(path)
    train_file_paths.extend(files)

val_file_paths = []

for path in val_paths:
    files = glob.glob(path)
    val_file_paths.extend(files)

test_file_paths = []

for path in test_paths:
    files = glob.glob(path)
    test_file_paths.extend(files)

In [4]:
train_dfs = []

for file in train_file_paths:
    df = pd.read_csv(file, sep='\t')
    df = df.iloc[:, 1:] # removing tweet ids
    df = df[df['class_label'] != 'missing_or_found_people']
    train_dfs.append(df)

train_df = pd.concat(train_dfs, ignore_index=True)

val_dfs = [] # dataframes

for file in val_file_paths:
    df = pd.read_csv(file, sep='\t')
    df = df.iloc[:, 1:] # removing tweet ids
    df = df[df['class_label'] != 'missing_or_found_people']
    val_dfs.append(df)

val_df = pd.concat(val_dfs, ignore_index=True)

test_dfs = [] # dataframes

for file in test_file_paths:
    df = pd.read_csv(file, sep='\t')
    df = df.iloc[:, 1:] # removing tweet ids
    df = df[df['class_label'] != 'missing_or_found_people']
    test_dfs.append(df)

test_df = pd.concat(test_dfs, ignore_index=True)

In [5]:
class_labels = train_df['class_label'].unique().tolist()
class_counts = train_df['class_label'].value_counts()
class_counts

class_label
rescue_volunteering_or_donation_effort    14891
other_relevant_information                 8501
sympathy_and_support                       6250
infrastructure_and_utility_damage          5715
injured_or_dead_people                     5110
not_humanitarian                           4407
caution_and_advice                         3774
displaced_people_and_evacuations           2800
requests_or_urgent_needs                   1833
Name: count, dtype: int64

Brute force Balance

In [6]:
# brute force balance
class_labels = train_df['class_label'].unique().tolist()
class_counts = train_df['class_label'].value_counts()
# Minimum class size
min_class_size = 3000 #min(class_counts)

balanced_dfs = []  # List to hold balanced DataFrames for each class
for class_label in class_labels:
    class_df = train_df[train_df['class_label'] == class_label]
    # Sample the minimum class size, if sample size is greater than the class size, then sample with replacement
    sampled_df = class_df.sample(min_class_size, replace=min_class_size > class_df.shape[0])
    balanced_dfs.append(sampled_df)

train_df = pd.concat(balanced_dfs, ignore_index=True)
# Optional: Shuffle the balanced DataFrame
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

class_labels = [s.strip() for s in class_labels]

In [7]:
class_labels = train_df['class_label'].unique().tolist()
class_counts = train_df['class_label'].value_counts()
class_counts

class_label
other_relevant_information                3000
displaced_people_and_evacuations          3000
sympathy_and_support                      3000
infrastructure_and_utility_damage         3000
not_humanitarian                          3000
caution_and_advice                        3000
requests_or_urgent_needs                  3000
rescue_volunteering_or_donation_effort    3000
injured_or_dead_people                    3000
Name: count, dtype: int64

In [8]:
id2label = {id:label for id, label in enumerate(class_labels)}
label2id = {label:id for id, label in enumerate(class_labels)}

In [9]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_df['class_label_num'] = label_encoder.fit_transform(train_df['class_label'])
val_df['class_label_num'] = label_encoder.transform(val_df['class_label'])
test_df['class_label_num'] = label_encoder.transform(test_df['class_label'])

In [10]:
train_df.head()

,tweet_text,class_label,class_label_num
0,The almighty Typhoon Haiyan struck the phillip...,other_relevant_information,5
1,"When it comes to hurricanes, preparation is ke...",displaced_people_and_evacuations,1
2,#earthquake Pray for safety of people plz,sympathy_and_support,8
3,Mother Nature is not to be trifled with -- wed...,infrastructure_and_utility_damage,2
4,And if u ever saw what I wrote here today I as...,not_humanitarian,4


In [11]:
train_tweets = list(train_df.tweet_text)
train_labels = list(train_df.class_label_num)

val_tweets = list(val_df.tweet_text)
val_labels = list(val_df.class_label_num)

test_tweets = list(test_df.tweet_text)
test_labels = list(test_df.class_label_num)

In [12]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(model_name)

In [13]:
train_encodings = tokenizer(train_tweets, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_tweets, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_tweets, truncation=True, padding=True, max_length=512)

In [14]:
class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)


In [15]:
train_dataset = TweetDataset(train_encodings, train_labels)  # Use resampled labels
val_dataset = TweetDataset(val_encodings, val_labels)
test_dataset = TweetDataset(test_encodings, test_labels)

In [16]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(class_labels))
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [17]:
training_args = TrainingArguments(
    output_dir='./content/bert_model_output',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    fp16=True,
    warmup_steps=200,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='./content/logs',
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

d:\AI_Dev_Vault\Anaconda\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [18]:
# Train the model
trainer.train()

  0%|          | 0/5064 [00:00<?, ?it/s]

{'loss': 2.1388, 'learning_rate': 2.425e-05, 'epoch': 0.06}


  0%|          | 0/485 [00:00<?, ?it/s]

{'eval_loss': 1.9613018035888672, 'eval_accuracy': 0.2351424519788578, 'eval_f1': 0.17561170333514334, 'eval_precision': 0.43537709271347547, 'eval_recall': 0.2351424519788578, 'eval_runtime': 23.9275, 'eval_samples_per_second': 324.188, 'eval_steps_per_second': 20.27, 'epoch': 0.06}
{'loss': 1.4228, 'learning_rate': 4.9250000000000004e-05, 'epoch': 0.12}


  0%|          | 0/485 [00:00<?, ?it/s]

...

  0%|          | 0/485 [00:00<?, ?it/s]

{'eval_loss': 0.9001975655555725, 'eval_accuracy': 0.754544282583473, 'eval_f1': 0.7541927395568432, 'eval_precision': 0.7615503003319412, 'eval_recall': 0.754544282583473, 'eval_runtime': 24.6439, 'eval_samples_per_second': 314.763, 'eval_steps_per_second': 19.68, 'epoch': 2.9}
{'loss': 0.239, 'learning_rate': 6.990131578947369e-07, 'epoch': 2.96}


  0%|          | 0/485 [00:00<?, ?it/s]

{'eval_loss': 0.8927129507064819, 'eval_accuracy': 0.7577671780327446, 'eval_f1': 0.7566188111542367, 'eval_precision': 0.7627854759109136, 'eval_recall': 0.7577671780327446, 'eval_runtime': 26.6062, 'eval_samples_per_second': 291.549, 'eval_steps_per_second': 18.229, 'epoch': 2.96}
{'train_runtime': 2420.6782, 'train_samples_per_second': 33.462, 'train_steps_per_second': 2.092, 'train_loss': 0.5103363904154508, 'epoch': 3.0}


TrainOutput(global_step=5064, training_loss=0.5103363904154508, metrics={'train_runtime': 2420.6782, 'train_samples_per_second': 33.462, 'train_steps_per_second': 2.092, 'train_loss': 0.5103363904154508, 'epoch': 3.0})

In [19]:
from sklearn.metrics import classification_report
predictions = trainer.predict(test_dataset)
labels = predictions.label_ids
preds = predictions.predictions.argmax(-1)

print(classification_report(labels, preds, target_names=class_labels))

  0%|          | 0/943 [00:00<?, ?it/s]

                                        precision    recall  f1-score   support

            other_relevant_information       0.56      0.81      0.66      1070
      displaced_people_and_evacuations       0.85      0.92      0.88       790
                  sympathy_and_support       0.83      0.78      0.80      1617
     infrastructure_and_utility_damage       0.92      0.93      0.93      1447
                      not_humanitarian       0.55      0.68      0.61      1245
                    caution_and_advice       0.64      0.37      0.47      2407
              requests_or_urgent_needs       0.39      0.78      0.52       521
rescue_volunteering_or_donation_effort       0.86      0.82      0.84      4219
                injured_or_dead_people       0.84      0.81      0.82      1772

                              accuracy                           0.75     15088
                             macro avg       0.72      0.77      0.73     15088
                          weighted avg

In [20]:
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

train_encodings = tokenizer(train_tweets, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_tweets, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_tweets, truncation=True, padding=True, max_length=512)

train_dataset = TweetDataset(train_encodings, train_labels)  # Use resampled labels
val_dataset = TweetDataset(val_encodings, val_labels)
test_dataset = TweetDataset(test_encodings, test_labels)

model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(class_labels))
model.to(device)

d:\AI_Dev_Vault\Anaconda\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dastan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [21]:
training_args = TrainingArguments(
    output_dir='./content/bert_model_output',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    fp16=True,
    warmup_steps=200,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='./content/logs',
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=2000,
    load_best_model_at_end=True,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

d:\AI_Dev_Vault\Anaconda\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


  0%|          | 0/5064 [00:00<?, ?it/s]

{'loss': 2.1419, 'learning_rate': 2.5e-05, 'epoch': 0.06}


  0%|          | 0/485 [00:00<?, ?it/s]

d:\AI_Dev_Vault\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

  0%|          | 0/485 [00:00<?, ?it/s]

{'eval_loss': 0.8854002356529236, 'eval_accuracy': 0.7487430707747841, 'eval_f1': 0.749469282906591, 'eval_precision': 0.7556845279348615, 'eval_recall': 0.7487430707747841, 'eval_runtime': 13.1889, 'eval_samples_per_second': 588.147, 'eval_steps_per_second': 36.773, 'epoch': 2.9}
{'loss': 0.2657, 'learning_rate': 6.887335526315789e-07, 'epoch': 2.96}


  0%|          | 0/485 [00:00<?, ?it/s]

{'eval_loss': 0.8810961842536926, 'eval_accuracy': 0.7490009024107258, 'eval_f1': 0.7490307792430112, 'eval_precision': 0.7542143741854624, 'eval_recall': 0.7490009024107258, 'eval_runtime': 13.2827, 'eval_samples_per_second': 583.991, 'eval_steps_per_second': 36.514, 'epoch': 2.96}
{'train_runtime': 1235.8153, 'train_samples_per_second': 65.544, 'train_steps_per_second': 4.098, 'train_loss': 0.5237516046135346, 'epoch': 3.0}


TrainOutput(global_step=5064, training_loss=0.5237516046135346, metrics={'train_runtime': 1235.8153, 'train_samples_per_second': 65.544, 'train_steps_per_second': 4.098, 'train_loss': 0.5237516046135346, 'epoch': 3.0})

In [22]:
from sklearn.metrics import classification_report
predictions = trainer.predict(test_dataset)
labels = predictions.label_ids
preds = predictions.predictions.argmax(-1)

print(classification_report(labels, preds, target_names=class_labels))

  0%|          | 0/943 [00:00<?, ?it/s]

                                        precision    recall  f1-score   support

            other_relevant_information       0.62      0.78      0.69      1070
      displaced_people_and_evacuations       0.82      0.93      0.88       790
                  sympathy_and_support       0.82      0.80      0.81      1617
     infrastructure_and_utility_damage       0.93      0.93      0.93      1447
                      not_humanitarian       0.53      0.70      0.61      1245
                    caution_and_advice       0.61      0.42      0.50      2407
              requests_or_urgent_needs       0.39      0.76      0.51       521
rescue_volunteering_or_donation_effort       0.88      0.78      0.83      4219
                injured_or_dead_people       0.82      0.83      0.82      1772

                              accuracy                           0.75     15088
                             macro avg       0.71      0.77      0.73     15088
                          weighted avg